# Perform A/B Test using REST Endpoints

<img src="img/sagemaker-architecture.png" width="80%" align="left">

In [ ]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r training_job_name

In [ ]:
print(training_job_name)

# Copy the Model to the Notebook

In [ ]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

In [ ]:
!tar -xvzf ./model.tar.gz

# Show the Prediction Signature

In [ ]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

In [ ]:
import boto3
client = boto3.client("sagemaker")

# Create Variant A Model From the Training Job in a Previous Section

Notes:
* `primary_container_image` is required because the inference and training images are different.
* By default, the training image will be used, so we need to override it.  
* See https://github.com/aws/sagemaker-python-sdk/issues/1379
* This variant requires the Elastic Inference image since we are using Elastic Inference
* If you are not using a US-based region, you may need to adapt the container image to your current region using the following table:

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

model_a_name = '{}-{}-{}'.format(training_job_name, 'eia', timestamp)

sess.create_model_from_job(name=model_a_name,
                           training_job_name=training_job_name,
                           role=role,
                           primary_container_image='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference-eia:2.0.0-cpu-py36-ubuntu18.04'.format(region))


# Create Variant B Model From the Training Job in a Previous Section
Notes:
* This is the same underlying model as variant A, but does not use an attached Elastic Inference Adapter (EIA).
* `primary_container_image` is required because the inference and training images are different.
* By default, the training image will be used, so we need to override it.  
* See https://github.com/aws/sagemaker-python-sdk/issues/1379
* If you are not using a US-based region, you may need to adapt the container image to your current region using the following table:

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

In [ ]:
model_b_name = '{}-{}-{}'.format(training_job_name, 'no-eia', timestamp)

sess.create_model_from_job(name=model_b_name,
                           training_job_name=training_job_name,
                           role=role,
                           primary_container_image='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference:2.1.0-cpu-py36-ubuntu18.04'.format(region))


# Canary Rollouts and A/B Testing

Instead of `deploy()`, we can create an `Endpoint Configuration` with multiple variants for canary rollouts and A/B testing.

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

endpoint_config_name = '{}-{}'.format(training_job_name, timestamp)

endpoint_config = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
         'VariantName': 'VariantA-WithElasticInference',
         'ModelName': model_a_name,
         'InstanceType':'ml.m4.xlarge',
         'InitialInstanceCount': 2,
         'InitialVariantWeight': 50,
         'AcceleratorType':'ml.eia2.medium' # This variant will use an Elastic Inference Adapter (GPU)            
        },
        {
         'VariantName': 'VariantB-WithoutElasticInference',
         'ModelName': model_b_name,
         'InstanceType':'ml.m4.xlarge',
         'InitialInstanceCount': 2,
         'InitialVariantWeight': 50,
        }
    ])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpointConfig/{}">REST Endpoint Configuration</a></b>'.format(region, endpoint_config_name)))


In [ ]:
endpoint_name = '{}-{}'.format(training_job_name, timestamp)

endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


# _Wait Until the ^^ Endpoint ^^ is Deployed_

In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Simulate a Prediction from an Application

## Setup the Request Handler to Convert Raw Text into BERT Tokens

In [ ]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)

## Setup the Response Handler to Convert the BERT Response into Our Predicted Classes

In [ ]:
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes

## Instantiate the Request/Response Handler Classes Above

In [ ]:
import json
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[1, 2, 3, 4, 5])

## Instantiate the Predictor with our Endpoint

In [ ]:
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

In [ ]:
import tensorflow as tf
import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# Review the REST Endpoint Performance Metrics

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint Performance Metrics</a></b>'.format(region, endpoint_name)))


# Shift All Traffic to Variant B
_**No downtime** occurs during this traffic-shift activity._

This may take a few minutes.  Please be patient.

In [ ]:
updated_endpoint_config = [
    {
        'VariantName': 'VariantA-WithElasticInference',
        'DesiredWeight': 0,
    },
    {
        'VariantName': 'VariantB-WithoutElasticInference',
        'DesiredWeight': 100,
    }
]

In [ ]:
client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=updated_endpoint_config
)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


# _Wait for the ^^ Endpoint Update ^^ to Complete Above_
This may take a few minutes.  Please be patient.

In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Remove Variant A to Reduce Cost
Modify the Endpoint Configuration to only use variant B.

_**No downtime** occurs during this scale-down activity._

This may take a few mins.  Please be patient.

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

updated_endpoint_config_name = '{}-{}'.format(training_job_name, timestamp)

updated_endpoint_config = client.create_endpoint_config(
    EndpointConfigName=updated_endpoint_config_name,
    ProductionVariants=[
        {
         'VariantName': 'VariantB-WithoutElasticInference',
         'ModelName': model_b_name,  # Only specify variant B to remove variant A
         'InstanceType':'ml.m4.xlarge',
         'InitialInstanceCount': 2,
         'InitialVariantWeight': 100
        }
    ])

In [ ]:
client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=updated_endpoint_config_name
)

# _If You See An ^^ Error ^^ Above, Please Wait Until the Endpoint is Updated_

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))
